In [1]:
!pip install sklearn_pandas
!pip install git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to /tmp/pip-ionbry2c-build
  Running setup.py install for sklearn2pmml ... - \ done


# Scikit-learn demo

## Build pipeline

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import urllib.parse
import json

from sklearn.datasets import load_diabetes,load_iris
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn2pmml import PMMLPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn2pmml import sklearn2pmml

diabetes = load_diabetes()
model = LinearRegression()
model.fit(diabetes.data, diabetes.target)
expected  = diabetes.target
predicted = model.predict(diabetes.data)
print("Mean Squared Error: %0.3f" % mse(expected, predicted))
print("Coefficient of Determination: %0.3f" % r2_score(expected, predicted))

iris = load_iris()
iris_df = pd.DataFrame(iris.data,columns=iris.feature_names)
iris_df['Species'] = iris.target
iris_pipeline = PMMLPipeline([
    ("classifier", DecisionTreeClassifier())
])
iris_pipeline.fit(iris_df[iris_df.columns.difference(["Species"])], iris_df["Species"])

Mean Squared Error: 2859.690
Coefficient of Determination: 0.518


PMMLPipeline(steps=[('classifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])

### Convert to PMML

In [3]:
sklearn2pmml(iris_pipeline, "DecisionTreeIris.pmml", with_repr = True)
pmmlBytes = bytearray(open('DecisionTreeIris.pmml', 'rb').read())

### Serve

In [ ]:
import urllib.request

namespace = 'default'
model_name = 'iris'
version = '1'
"""
prediction-pmml IP address can be found via: kubectl describe svc prediction-pmml
"""
update_url = 'http://<prediction-pmml service IP>/update-pmml/%s/%s/%s' % (namespace, model_name, version)

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = urllib.request.Request(update_url, \
                             headers=update_headers, \
                             data=pmmlBytes)

resp = urllib.request.urlopen(req, timeout=5)

print(resp.status) # Should return Http Status 200

In [8]:
!

kubectl controls the Kubernetes cluster manager. 

Find more information at https://github.com/kubernetes/kubernetes.

Basic Commands (Beginner):
  create         Create a resource by filename or stdin
  expose         Take a replication controller, service, deployment or pod and
expose it as a new Kubernetes Service
  run            Run a particular image on the cluster
  set            Set specific features on objects

Basic Commands (Intermediate):
  get            Display one or many resources
  explain        Documentation of resources
  edit           Edit a resource on the server
  delete         Delete resources by filenames, stdin, resources and names, or
by resources and label selector

Deploy Commands:
  rollout        Manage a deployment rollout
  rolling-update Perform a rolling update of the given ReplicationController
  scale          Set a new size for a Deployment, ReplicaSet, Replication
Controller, or Job
  autoscale      Auto-scale a Deployment, ReplicaSet, or Repli

### Evaluate

In [ ]:
import urllib.parse
import json

namespace = 'default'
model_name = 'iris'
version = '1'
evaluate_url = 'http://prediction-pmml service IP/evaluate-pmml/%s/%s/%s' % (namespace, model_name, version)

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'

test = iris_df
input_params = test.ix[0,:-1].to_json()
encoded_input_params = input_params.encode('utf-8')
req = urllib.request.Request(evaluate_url, headers=evaluate_headers,data=encoded_input_params)
resp = urllib.request.urlopen(req, timeout=3)
print(resp.read().decode())